<a href="https://colab.research.google.com/github/AMohamedAakhil/kaggle-housing-price-prediction/blob/main/kaggle_housingRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
train_data = pd.read_csv('train (3).csv')
test_data = pd.read_csv('test (3).csv')



In [ ]:
test_data.isna().sum()

Id                 0
MSSubClass         0
MSZoning           4
LotFrontage      227
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 80, dtype: int64

In [ ]:
test_data.MSZoning.fillna('RL', inplace=True)
train_data.MSZoning.fillna('RL', inplace=True)


In [ ]:
test_data.drop('Alley', axis=1, inplace=True)
train_data.drop('Alley', axis=1, inplace=True)

In [ ]:
test_data.Utilities = test_data.Utilities.dropna()
test_data.Utilities = train_data.Utilities.dropna()

In [ ]:
test_data.drop('MiscFeature', axis=1, inplace=True)
train_data.drop('MiscFeature', axis=1, inplace=True)

test_data.drop('PoolQC', axis=1, inplace=True)
train_data.drop('PoolQC', axis=1, inplace=True)

test_data.drop('Fence', axis=1, inplace=True)
train_data.drop('Fence', axis=1, inplace=True)



In [ ]:
train_data.BsmtFinType1


0       GLQ
1       ALQ
2       GLQ
3       ALQ
4       GLQ
       ... 
1455    Unf
1456    ALQ
1457    GLQ
1458    GLQ
1459    BLQ
Name: BsmtFinType1, Length: 1460, dtype: object

In [ ]:
train_data.FireplaceQu.fillna('Gd', inplace=True)
test_data.FireplaceQu.fillna('Gd', inplace=True)

In [ ]:
train_data = train_data.dropna()


In [ ]:
test_data.isna().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      227
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 76, dtype: int64

In [ ]:
train_data.MSZoning.unique()

array(['RL', 'RM', 'C (all)', 'FV', 'RH'], dtype=object)

In [ ]:
train_data.append(pd.get_dummies(train_data.MSZoning))
train_data.drop('MSZoning', axis=1, inplace=True)

In [ ]:
train_data.head()

,Id,MSSubClass,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [ ]:
test_data.append(pd.get_dummies(test_data.MSZoning))
test_data.drop('MSZoning', axis=1, inplace=True)



In [ ]:
test_data.shape, train_data.shape

((1459, 75), (1094, 76))

In [ ]:
catdf_train = train_data.select_dtypes(include='object')
catdf_test = test_data.select_dtypes(include='object')

In [ ]:
catdf_train.shape, catdf_test.shape

((1094, 38), (1459, 38))

In [ ]:
catdf_train_ohe = pd.get_dummies(catdf_train)
catdf_test_ohe = pd.get_dummies(catdf_test)


In [ ]:


# columns to drop
cols_to_drop = set(catdf_test_ohe.columns).difference(set(catdf_train_ohe.columns))

# drop columns
catdf_test_ohe.drop(cols_to_drop, axis=1, inplace=True)

catdf_train_ohe.shape, catdf_test_ohe.shape


((1094, 223), (1459, 212))

In [ ]:
test_data.drop(catdf_test, axis=1, inplace=True)
train_data.drop(catdf_train, axis=1, inplace=True)

In [ ]:
test_data = pd.concat([test_data, catdf_test_ohe], axis=1)
train_data = pd.concat([train_data, catdf_train_ohe], axis=1)

In [ ]:
noidtest = test_data

from sklearn.impute import SimpleImputer
 
imputer = SimpleImputer(missing_values = np.nan, strategy ='mean') 
# Imputing the data    
noidtest = imputer.fit_transform(noidtest)
noidtest = pd.DataFrame(noidtest)

X = train_data.drop('SalePrice', axis=1)
y = train_data.SalePrice
X = imputer.fit_transform(X)
X = pd.DataFrame(X)
# cols_to_drop1 = set(noidtest.columns).difference(set(X.columns))
# noidtest.drop(cols_to_drop1, axis=1, inplace=True)


cols_to_drop2 = set(X.columns).difference(set(noidtest.columns))
X.drop(cols_to_drop2, axis=1, inplace=True)
X, noidtest




x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.35)

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor as xgbr 
from sklearn.metrics import mean_squared_error as MSE


  
xgb_r = xgbr(n_estimators=2000,learning_rate=0.055)
pred = xgb_r.fit(X, y)
ypreds = xgb_r.predict(noidtest)  
# RMSE Computation


[11:22:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
model.score(x_test, y_test)

In [ ]:
# GridSearchCV goes here

In [ ]:
test_dmatrix = xg.DMatrix(data = noidtest, label = y_test)

ypreds = xgb_r.predict(test_dmatrix)

In [ ]:
ypreds

array([102615.82 ,  81783.67 , 176555.88 , ..., 159861.4  , 116617.055,
       165459.69 ], dtype=float32)

In [ ]:
test_data1 = pd.read_csv('test (3).csv')
sumbit = {'Id' : test_data1.Id, 'SalePrice': ypreds}

submission_df = pd.DataFrame(sumbit, columns={'Id': test_data.Id, 'SalePrice': ypreds})

In [ ]:
submission_df.to_csv('submission4.csv', index=False)

In [ ]:
len(submission_df)

1459